In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Daily EOD Scheduler — Supertrend(10,3) + RSI(14) + RS(55) + optional EMA200

How it works (EOD cadence):
1) Loads persistent state (open positions, pending orders) from ./state/positions.json
2) Fetches OHLCV for your universe + benchmark (yfinance, auto_adjust=True).
3) Promotes any PENDING orders whose planned date has arrived:
     - BUY: fill at that session's Open -> becomes an open position; SL/TP set.
     - SELL: fill at that session's Open -> position closed; execution recorded.
4) On the most recent completed bar (D0), generates:
     - New BUY signals (for next session open), unless already held or pending.
     - Exit signals (for next session open) for currently held positions:
         • If D0 High ≥ TP or Low ≤ SL  ⇒ schedule SELL next open (priority configurable)
         • Else if indicator exit (ST red or RS<0 or RSI<60) ⇒ schedule SELL next open.
5) Saves:
     - ./outputs/plan_YYYYMMDD.csv   (orders to place next session)
     - ./outputs/executions.csv       (append-only fills history)
     - ./outputs/open_positions.csv   (snapshot of current holdings)
     - ./state/positions.json         (persistent state)
"""

import os, json, math, warnings, logging
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%-Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("daily_scheduler_supertrend_rsi_rs55")

# =========================
# CONFIG (no CLI; edit here)
# =========================
@dataclass
class Config:
    # Data window
    start_date: str = "2015-01-01"
    end_date: Optional[str] = None  # None = today India
    max_lookback_days: int = 600    # fetch recent to speed up

    # Universe ('.NS' suffix expected; add more tickers or set from file)
    static_symbols: List[str] = None
    static_symbols_path: Optional[str] = None

    # Benchmark for RS(55) Mansfield (tries these in order)
    benchmark_try: Tuple[str,...] = ("^CNX500","^NIFTY500","^CRSLDX","^BSE500","^NSE500","^NSEI")

    # Indicators / Strategy
    rsi_len: int = 14
    rs_lookback: int = 55
    st_atr_len: int = 10
    st_mult: float = 3.0
    use_ema200_filter: bool = True
    ema200_len: int = 200

    # Entry/Exit thresholds
    rsi_entry_min: float = 60.0
    rsi_exit_min: float  = 60.0
    rs_zero: float = 0.0  # >0 to enter, <0 to exit

    # Risk management
    stop_loss_pct: float = 0.05   # 5% hard stop
    target_pct: float    = 0.10   # 10% hard target
    stop_target_priority: str = "stop_first"  # or "target_first"

    # Portfolio sizing (simple)
    capital_inr: float = 200_000.0
    per_trade_inr: float = 50_000.0  # fixed allocation per new entry
    max_concurrent_positions: int = 4
    top_k_daily: int = 5             # cap number of new entries per day

    # Optional universe filters for new entries
    within_pct_of_52w_high: float = 0.70
    filter_52w_window: int = 252
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0
    enable_basic_liquidity: bool = False

    # Paths
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    state_dir: str = "state"

CFG = Config(
    static_symbols= ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']

)

# =========================
# FEES (simple, optional)
# =========================
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    # Keep light for scheduler; edit to your broker if needed
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001; STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297; SEBI_PCT = 0.000001; IPFT_PCT = 0.000001
    GST_PCT = 0.18; DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = max(BROKER_MIN, min(turnover * BROKER_PCT, BROKER_CAP))
        return fee

    br_buy, br_sell = _broker(turnover_buy), _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    gst   = GST_PCT * (br_buy + br_sell + exch + sebi + ipft + DP_SELL)
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + DP_SELL + gst)

# =========================
# IO helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_india_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

STATE_PATH = os.path.join(CFG.state_dir, "positions.json")
EXEC_PATH  = os.path.join(CFG.out_dir, "executions.csv")

def load_state():
    ensure_dirs(CFG.state_dir, CFG.out_dir, CFG.cache_dir)
    if os.path.exists(STATE_PATH):
        with open(STATE_PATH, "r") as f:
            return json.load(f)
    return {"cash_inr": CFG.capital_inr, "open_positions": {}, "pending_orders": []}

def save_state(state):
    with open(STATE_PATH, "w") as f:
        json.dump(state, f, indent=2, default=str)

def append_execution_row(row: dict):
    ensure_dirs(CFG.out_dir)
    row = {k: (v if not isinstance(v, pd.Timestamp) else v.strftime("%Y-%m-%d")) for k, v in row.items()}
    if not os.path.exists(EXEC_PATH):
        pd.DataFrame([row]).to_csv(EXEC_PATH, index=False)
    else:
        pd.DataFrame([row]).to_csv(EXEC_PATH, mode="a", header=False, index=False)

# =========================
# Data + Indicators
# =========================
def _normalize_syms(symbols: List[str]) -> List[str]:
    out = []
    for s in symbols:
        s = s.strip().upper()
        if not s.endswith(".NS") and not s.startswith("^"):
            s = f"{s}.NS"
        out.append(s)
    return out

def fetch_prices(tickers: List[str], start: str, end: Optional[str]) -> Dict[str, pd.DataFrame]:
    ensure_dirs(CFG.cache_dir)
    data = {}
    end = end or today_india_str()
    for ticker in tickers:
        cache_path = os.path.join(CFG.cache_dir, f"{ticker.replace('^','_')}.parquet")
        use_cached = False
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    use_cached = True
            except Exception:
                pass
        if not use_cached:
            try:
                df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
                if df is None or df.empty:
                    continue
                df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
                df.index.name = "date"
                df = df.tail(CFG.max_lookback_days)
                df.to_parquet(cache_path)
                data[ticker] = df
            except Exception:
                continue
    return data

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    d = series.diff()
    gain = d.clip(lower=0).rolling(length).mean()
    loss = (-d.clip(upper=0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)

def _true_range(h, l, pc):
    return pd.concat([(h-l).abs(), (h-pc).abs(), (l-pc).abs()], axis=1).max(axis=1)

def atr(h, l, c, length=14):
    return _true_range(h, l, c.shift(1)).ewm(alpha=1/length, adjust=False, min_periods=length).mean()

def supertrend(df: pd.DataFrame, atr_len: int, mult: float) -> pd.DataFrame:
    h, l, c = df["High"], df["Low"], df["Close"]
    _atr = atr(h, l, c, atr_len)
    hl2 = (h + l) / 2.0
    ub = hl2 + mult * _atr
    lb = hl2 - mult * _atr
    st = pd.Series(index=df.index, dtype=float)
    dr = pd.Series(index=df.index, dtype=int)
    st.iloc[0] = ub.iloc[0]; dr.iloc[0] = -1
    for i in range(1, len(df)):
        up = min(ub.iloc[i], ub.iloc[i-1])
        lo = max(lb.iloc[i], lb.iloc[i-1])
        if dr.iloc[i-1] == -1:
            st_val = up
            if c.iloc[i] > st_val:
                dr.iloc[i] = 1; st_val = lo
            else:
                dr.iloc[i] = -1
        else:
            st_val = lo
            if c.iloc[i] < st_val:
                dr.iloc[i] = -1; st_val = up
            else:
                dr.iloc[i] = 1
        st.iloc[i] = st_val
    out = df.copy()
    out["st"] = st; out["st_dir"] = dr.fillna(-1)
    return out[["st","st_dir"]]

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str]) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_india_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def mansfield_rs(stock_close: pd.Series, bench_close: pd.Series, lookback: int) -> pd.Series:
    r = (stock_close / bench_close).dropna()
    sma = r.rolling(lookback).mean()
    rs = 100.0 * (r / sma - 1.0)
    return rs.reindex(stock_close.index).fillna(method="ffill")

def compute_indicators(df: pd.DataFrame, bench_df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["rsi"] = rsi(out["Close"], CFG.rsi_len)
    st = supertrend(out[["High","Low","Close"]], CFG.st_atr_len, CFG.st_mult)
    out = pd.concat([out, st], axis=1)
    out["ema200"] = ema(out["Close"], CFG.ema200_len)
    # RS vs benchmark
    bclose = bench_df["Close"].reindex(out.index).fillna(method="ffill")
    out["rs55"] = mansfield_rs(out["Close"], bclose, CFG.rs_lookback)
    # helpers
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"] = out["Close"].rolling(CFG.filter_52w_window).max()
    return out.dropna()

# =========================
# Signal Logic (EOD bar D0 → plan for next session)
# =========================
def basic_liquidity_ok(row: pd.Series) -> bool:
    if not CFG.enable_basic_liquidity: return True
    if row["Close"] < CFG.min_price_inr: return False
    if row["avg_vol_20"] < CFG.min_avg_vol_20d: return False
    return True

def entry_conditions(row: pd.Series) -> Tuple[bool, str]:
    conds = []
    ok = True

    rsi_ok = row["rsi"] > CFG.rsi_entry_min
    conds.append(f"RSI>{CFG.rsi_entry_min:.0f}" if rsi_ok else f"RSI≤{CFG.rsi_entry_min:.0f}")
    ok &= rsi_ok

    rs_ok = row["rs55"] > CFG.rs_zero
    conds.append("RS55>0" if rs_ok else "RS55≤0")
    ok &= rs_ok

    st_flip = (row["st_dir"] == 1)  # we will check flip vs prior bar outside
    conds.append("ST=BUY" if st_flip else "ST≠BUY")
    ok &= st_flip

    if CFG.use_ema200_filter:
        ema_ok = row["Close"] > row["ema200"]
        conds.append("Close>EMA200" if ema_ok else "Close≤EMA200")
        ok &= ema_ok

    return bool(ok), "; ".join(conds)

def exit_conditions(row_today: pd.Series, st_red: bool, rs_below: bool, rsi_below: bool) -> Tuple[bool, str]:
    reasons = []
    if st_red:    reasons.append("Supertrend RED")
    if rs_below:  reasons.append("RS55<0")
    if rsi_below: reasons.append(f"RSI<{CFG.rsi_exit_min:.0f}")
    return (len(reasons) > 0), " & ".join(reasons)

# =========================
# State promotion (pending → executed)
# =========================
def promote_pending_orders(state, data_map):
    """
    For each pending order, if a bar exists on or after planned_date for that ticker,
    execute at that day's Open and update state.
    """
    still_pending = []
    for od in state["pending_orders"]:
        tkr = od["ticker"]
        df = data_map.get(tkr)
        if df is None or df.empty:
            still_pending.append(od)
            continue

        planned = pd.to_datetime(od["planned_date"]).date()
        # Find first available bar >= planned_date
        exec_date = None
        for dt in df.index.date:
            if dt >= planned:
                exec_date = pd.Timestamp(dt)
                break
        if exec_date is None:
            still_pending.append(od)
            continue

        px = float(df.loc[exec_date, "Open"])
        side = od["side"].upper()

        if side == "BUY":
            if tkr in state["open_positions"]:
                # Already held—skip
                continue
            # shares via fixed per-trade budget
            cap = float(state.get("cash_inr", 0.0))
            budget = min(CFG.per_trade_inr, cap)
            shares = int(math.floor(budget / px)) if px > 0 else 0
            if shares <= 0:
                log.info("Skip BUY fill %s — insufficient cash @ %.2f", tkr, px)
                continue
            turn = shares * px
            fee = calc_fees(turn, 0.0)
            total = turn + fee
            if total > cap:
                shares = int(math.floor((cap - fee) / px))
                if shares <= 0:
                    log.info("Skip BUY fill %s — cash after fees not enough", tkr)
                    continue
                turn = shares * px; total = turn + fee

            state["cash_inr"] = cap - total
            sl = px * (1 - CFG.stop_loss_pct)
            tp = px * (1 + CFG.target_pct)
            state["open_positions"][tkr] = {
                "entry_date": exec_date.strftime("%Y-%m-%d"),
                "entry_px": px,
                "shares": shares,
                "stop_px": sl,
                "target_px": tp
            }
            append_execution_row({
                "date": exec_date, "ticker": tkr, "side": "BUY",
                "price": px, "shares": shares, "fees_inr": fee,
                "note": od.get("reason","planned_buy")
            })
            log.info("FILLED BUY %-12s @ %.2f x %d | cash=%.2f | SL=%.2f TP=%.2f",
                     tkr, px, shares, state["cash_inr"], sl, tp)

        elif side == "SELL":
            pos = state["open_positions"].get(tkr)
            if pos is None:
                continue
            shares = int(pos["shares"])
            turn = shares * px
            fee = calc_fees(0.0, turn)
            pnl = (px - pos["entry_px"]) * shares - fee
            state["cash_inr"] = float(state.get("cash_inr", 0.0)) + (turn - fee)
            append_execution_row({
                "date": exec_date, "ticker": tkr, "side": "SELL",
                "price": px, "shares": shares, "fees_inr": fee,
                "note": od.get("reason","planned_sell"), "pnl_inr": pnl
            })
            del state["open_positions"][tkr]
            log.info("FILLED SELL %-12s @ %.2f x %d | pnl=%.2f | cash=%.2f",
                     tkr, px, shares, pnl, state["cash_inr"])
        else:
            still_pending.append(od)

    state["pending_orders"] = still_pending

# =========================
# Daily planning
# =========================
def next_bday(date_like: pd.Timestamp) -> str:
    # naive next business day (calendar) — actual fill is chosen from real bars on promotion
    d1 = pd.Timestamp(date_like).tz_localize(None) + pd.tseries.offsets.BDay(1)
    return d1.strftime("%Y-%m-%d")

def generate_daily_plan(state, data_map, bench_df) -> pd.DataFrame:
    """
    Inspects the LAST available bar D0 for each ticker to decide entries/exits
    to plan for next session open.
    """
    rows = []
    # Build quick views
    open_tickers = set(state["open_positions"].keys())
    pending_pairs = {(od["ticker"], od["side"]) for od in state["pending_orders"]}

    # For capacity management
    slots_avail = max(0, CFG.max_concurrent_positions - len(open_tickers))

    # ENTRY candidates (collect first, then select top-K by simple momentum score)
    entries = []
    indicators_map = {}
    for tkr, df in data_map.items():
        if df is None or df.empty: continue
        ind = compute_indicators(df, bench_df)
        if ind.empty: continue
        D0 = ind.index[-1]
        prev = ind.index[-2] if len(ind) >= 2 else None
        row = ind.iloc[-1]
        indicators_map[tkr] = (D0, row)

        # Skip if already held or pending buy
        if tkr in open_tickers or (tkr, "BUY") in pending_pairs:
            continue

        # Optional proximity to 52w high + basic liquidity
        if CFG.within_pct_of_52w_high > 0:
            hi = row["high_52w"]; cl = row["Close"]
            if not (hi > 0 and cl >= CFG.within_pct_of_52w_high * hi):
                continue
        if not basic_liquidity_ok(row):
            continue

        # Entry rules require a flip to BUY today: st_dir==1 and yesterday == -1
        st_flip_today = bool(row["st_dir"] == 1 and (ind.loc[prev, "st_dir"] == -1) if prev is not None else False)
        if not st_flip_today:
            continue

        ok, reason = entry_conditions(row)
        if not ok:
            continue

        # score: combine RS and RSI (simple)
        score = float(row["rs55"]) + float(row["rsi"])
        entries.append((tkr, D0, score, reason))

    # Pick top-K entries given available slots
    entries = sorted(entries, key=lambda x: x[2], reverse=True)
    entries = entries[:min(CFG.top_k_daily, slots_avail)]

    for (tkr, D0, _score, reason) in entries:
        # plan for next business day (actual fill chosen on promotion)
        rows.append({
            "planned_date": next_bday(D0),
            "side": "BUY",
            "ticker": tkr,
            "reason": f"{reason}; plan entry next open",
        })
        state["pending_orders"].append({
            "planned_date": next_bday(D0),
            "side": "BUY",
            "ticker": tkr,
            "reason": reason
        })

    # EXIT checks for open positions
    for tkr, pos in list(state["open_positions"].items()):
        df = data_map.get(tkr)
        if df is None or df.empty: continue
        ind = compute_indicators(df, bench_df)
        if ind.empty: continue
        row = ind.iloc[-1]
        D0 = ind.index[-1]
        # hard SL/TP check with D0 range
        sl = float(pos["stop_px"]); tp = float(pos["target_px"])
        lowD0 = float(ind.loc[D0, "Low"]); highD0 = float(ind.loc[D0, "High"])

        hit = None
        if CFG.stop_target_priority == "stop_first":
            if lowD0 <= sl:   hit = ("SELL", f"Hard SL hit ({sl:.2f}) on D0")
            elif highD0 >= tp: hit = ("SELL", f"Hard TP hit ({tp:.2f}) on D0")
        else:
            if highD0 >= tp: hit = ("SELL", f"Hard TP hit ({tp:.2f}) on D0")
            elif lowD0 <= sl:   hit = ("SELL", f"Hard SL hit ({sl:.2f}) on D0")

        if hit is None:
            st_red   = bool(row["st_dir"] == -1)
            rs_below = bool(row["rs55"] < CFG.rs_zero)
            rsi_below= bool(row["rsi"]  < CFG.rsi_exit_min)
            do_exit, why = exit_conditions(row, st_red, rs_below, rsi_below)
            if do_exit:
                hit = ("SELL", why)

        if hit is not None:
            side, why = hit
            if (tkr, side) not in pending_pairs:
                rows.append({
                    "planned_date": next_bday(D0),
                    "side": side,
                    "ticker": tkr,
                    "reason": f"{why}; plan exit next open",
                })
                state["pending_orders"].append({
                    "planned_date": next_bday(D0),
                    "side": side,
                    "ticker": tkr,
                    "reason": why
                })

    # Also emit a snapshot of open positions
    open_rows = []
    for tkr, pos in state["open_positions"].items():
        open_rows.append({
            "ticker": tkr,
            "entry_date": pos["entry_date"],
            "entry_px": pos["entry_px"],
            "shares": pos["shares"],
            "stop_px": pos["stop_px"],
            "target_px": pos["target_px"],
        })

    # Write outputs
    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d")
    plan_path = os.path.join(CFG.out_dir, f"plan_{stamp}.csv")
    open_path = os.path.join(CFG.out_dir, f"open_positions.csv")

    plan_df = pd.DataFrame(rows, columns=["planned_date","side","ticker","reason"]).sort_values(
        by=["planned_date","side","ticker"]
    )
    if plan_df.empty:
        # ensure file exists to inspect in automations
        pd.DataFrame(columns=["planned_date","side","ticker","reason"]).to_csv(plan_path, index=False)
    else:
        plan_df.to_csv(plan_path, index=False)

    pd.DataFrame(open_rows, columns=["ticker","entry_date","entry_px","shares","stop_px","target_px"]).to_csv(open_path, index=False)

    log.info("Wrote plan → %s", plan_path)
    log.info("Open positions snapshot → %s", open_path)
    return plan_df

# =========================
# Main
# =========================
def main():
    ensure_dirs(CFG.out_dir, CFG.state_dir, CFG.cache_dir)

    # Universe
    symbols = []
    if CFG.static_symbols_path and os.path.exists(CFG.static_symbols_path):
        with open(CFG.static_symbols_path, "r") as f:
            symbols = [ln.strip() for ln in f if ln.strip()]
    else:
        symbols = CFG.static_symbols or []
    symbols = _normalize_syms(symbols)

    if not symbols:
        raise RuntimeError("No symbols configured. Populate CFG.static_symbols or CFG.static_symbols_path.")

    # Data
    start = CFG.start_date
    end   = CFG.end_date or today_india_str()
    data_map = fetch_prices(symbols, start, end)
    bench_tkr, bench_df = pick_benchmark(CFG.benchmark_try, start, end)

    # Load & promote state
    state = load_state()
    promote_pending_orders(state, data_map)

    # Capacity guard
    if len(state["open_positions"]) > CFG.max_concurrent_positions:
        log.warning("Currently %d positions open > max %d. Scheduler will still generate exits but limit new entries.",
                    len(state["open_positions"]), CFG.max_concurrent_positions)

    # Plan for next session
    plan_df = generate_daily_plan(state, data_map, bench_df)

    # Persist state
    save_state(state)

    # Log summary
    log.info("Cash: %.2f | Open positions: %d | Pending orders: %d | New plan rows: %d",
             float(state.get("cash_inr", 0.0)), len(state["open_positions"]), len(state["pending_orders"]), len(plan_df))

if __name__ == "__main__":
    main()


2025-10-14 19:20:17 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ^CNX500"}}}
2025-10-14 19:20:18 | ERROR | 
1 Failed download:
2025-10-14 19:20:18 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-14 19:20:19 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ^NIFTY500"}}}
2025-10-14 19:20:19 | ERROR | 
1 Failed download:
2025-10-14 19:20:19 | ERROR | ['^NIFTY500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-14 19:20:19 | INFO | Using benchmark: ^CRSLDX
2025-10-14 19:20:29 | INFO | Wrote plan → outputs/plan_20251014.csv
2025-10-14 19:20:29 | INFO | Open positions snapshot → outputs/open_positions.csv
2025-10-14 19:20:29 | INFO | Cash: 200000.00 | Open positions: 0 | Pending orders: 0 | New plan rows: 0
